# Examples

## Loading and navigating the model
We start with importing the euromod package and creating a `Model` object from a EUROMOD model.

In [8]:
from euromod import Model
mod=Model(r"C:\EUROMOD_RELEASES_I6.0+")
mod

------------------------------
Model
------------------------------
	 countries: 28 elements
	 extensions: 11 elements
	 model_path: 'C:\\EUROMOD_RELEASES_I6.0+'

Note that every object that is related to the EUROMOD project comes with an informative description. Here we can see that the model has 3 relevant attributes to the user:
- countries
- extensions
- model_path
The countries and extensions attributes say that they contain elements. If we take a look at countries

In [9]:
mod.countries

0: AT 
1: BE 
2: BG 
3: CY 
4: CZ 
5: DE 
6: DK 
7: EE 
8: EL 
9: ES 
10: FI
11: FR
12: HR
13: HU
14: IE
15: IT
16: LT
17: LU
18: LV
19: MT
20: NL
21: PL
22: PT
23: RO
24: SE
25: SI
26: SK
27: SL

We see indeed that the euromod model contains 28 countries. In a similar fashion we could have looked what kind of extensions that are stored in the model. The countries container can be indexed by both the number of the element and the country shortcode. Let us take a look at Sweden.

In [10]:
mod.countries["SE"]

------------------------------
Country
------------------------------
	 datasets: 27 elements
	 extensions: 12 elements
	 local_extensions: COVID
	 name: 'SE'
	 policies: 26 elements
	 systems: 18 elements

In [11]:
mod.countries["SE"].policies[0:10]

0: setdefault_se     |                                  |    DEF: SET DEFAULT 
1: uprate_se         |                                  |    DEF: UPRATING FACTORS 
2: ConstDef_se       |                                  |    DEF: CONSTANTS 
3: IlsDef_se         |                                  |    DEF: INCOME CONCEPTS (standardized) 
4: IlsUDBdef_se      |                                  |    DEF: INCOME CONCEPTS (UDB) 
5: ildef_se          |                                  |    DEF: INCOME CONCEPTS (non-standardized) 
6: random_se         |                                  |    DEF: Random assignment 
7: TransLMA_se       |                                  |    DEF: Modelling labour market transitions (DO NOT S ... 
8: tudef_se          |                                  |    DEF: ASSESSMENT UNITS 
9: yem_se            |  (with switch set for MWA)       |    DEF: minimum wage 

## Running a system with default configuration
Say that we are interested in running the tax system for the year 2021 of Sweden. Building further on the previous example we can look at the tax-systems contained in the model for Sweden.

In [12]:
mod.countries["SE"].systems

0: SE_2006 
1: SE_2007 
2: SE_2008 
3: SE_2009 
4: SE_2010 
5: SE_2011 
6: SE_2012 
7: SE_2013 
8: SE_2014 
9: SE_2015 
10: SE_2016
11: SE_2017
12: SE_2018
13: SE_2019
14: SE_2020
15: SE_2021
16: SE_2022
17: SE_2023

In order to run the tax system we need a dataset that fits the requirement to use. The model however provides us with a list of datasets that are configured already.

In [13]:
mod.countries["SE"].systems["SE_2021"].datasets

0: training_data     |             
1: SE_2019_a1        |             
2: SE_2020_b1        |             
3: SE_2021_hhot      |             
4: SE_2021_b1        | best match  

Here we see that there are multiple datasets configured, but that PL_2020_b2 is seen as the best match dataset for this taxsystem. Provided that you have the microdata stored somewhere, you can then load it as a `pandas.DataFrame` and run the model in the following way:

In [14]:
import pandas as pd
data=pd.read_csv(r"C:\EUROMOD_RELEASES_I6.0+\Input\SE_2021_b1.txt",sep="\t")
out = mod.countries["SE"].systems["SE_2021"].run(data,"SE_2021_b1")
out

Simulation for system SE_2021 with dataset SE_2021_b1 finished.


------------------------------
Simulation
------------------------------
	 constantsToOverwrite: {}
	 errors: []
	 output_filenames: ['se_2021_std.txt']
	 outputs: Pandas DataFrame of 240 variables and 21671 observations.

In [15]:
pd.set_option('display.max_rows', 11, 'display.max_columns',7)

This returned us a `Simulation` object with multiple attributes returned. The one of interest here is outputs, which contains the outputdataset(s) returned by the microsimulation model.

In [16]:
outputdata_baseline = out.outputs[0]
outputdata_baseline

idhh     idperson     idmother  ...  tu_bho_se_IsDependentChild  \
0          200.0      20001.0          0.0  ...                         0.0   
1          300.0      30001.0          0.0  ...                         0.0   
2          300.0      30002.0          0.0  ...                         0.0   
3          500.0      50001.0          0.0  ...                         0.0   
4          500.0      50002.0          0.0  ...                         0.0   
...          ...          ...          ...  ...                         ...   
21666  1936500.0  193650002.0          0.0  ...                         0.0   
21667  1936800.0  193680001.0          0.0  ...                         0.0   
21668  1936800.0  193680002.0          0.0  ...                         0.0   
21669  1936800.0  193680003.0  193680002.0  ...                         1.0   
21670  1936800.0  193680004.0  193680002.0  ...                         1.0   

       tu_bho_se_IsLoneParent  tu_bho_se_IsPartner  
0                         0.0                  0.0  
1                         0.0                  0.0  
2                         0.0                  1.0  
3                         0.0                  0.0  
4                         0.0                  1.0  
...                       ...                  ...  
21666                     0.0                  1.0  
21667                     0.0                  1.0  
21668                     0.0                  0.0  
21669                     0.0                  0.0  
21670                     0.0                  0.0  

[21671 rows x 240 columns]

## Running a system while changing a constant
One of the advantages of using the Python Connectors is the ability to run many counterfactual scenario's for the EUROMOD model. One can for example change the Tax Free income limit in Poland. There are multiple ways to do this via the euromod package in Python, but one very straightforward way is to use the constantsToOverwrite option which is a a dictionary, having the targetted constant as a key and the value to overwrite with as a value.


In [17]:
out=mod['SE']['SE_2021'].run(data,"SE_2021_b1",constantsToOverwrite={("$tinna_rate2",""):'0.4'})
outputdata_changed= out.outputs[0]
sum(outputdata_changed.ils_dispy - outputdata_baseline.ils_dispy)

Simulation for system SE_2021 with dataset SE_2021_b1 finished.


-12247774.761818277

The optional parameter ```constantsToOverwrite``` specifies which constants to overwrite in the policy spline. `constantsToOverwrite` must be  a ```dict```, where the keys are ```tuples``` of two ```str``` objects: the first string is the name of the constant and the second string is its group number (**Note:** Pass an empty string if the group number is `None`); the values are ```str``` with the new values of the constants. The default is None.

## Run with add-ons
Run the simulation for the Swedish system SE_2022 including the Marginal Tax-Rate add-on 'MTR'.

In [18]:
out =mod['SE']['SE_2022'].run(data,"SE_2021_b1",addons=[("MTR","MTR")])
out

Simulation for system SE_2022 with dataset SE_2021_b1 finished.


------------------------------
Simulation
------------------------------
	 constantsToOverwrite: {}
	 errors: []
	 output_filenames: ['se_2022_base_mtr.txt', 'se_2022_mtr.txt']
	 outputs: Pandas DataFrame of 246 variables and 21671 observations., Pandas DataFrame of 38 variables and 21671 observations.

As one can see there are two datasets returned by the model. Both of them can be accessed. The average marginal tax rate for example can then be straightforwardly computed as

In [19]:
out.outputs['se_2022_mtr.txt'].mtrpc.mean()

19.551933352523093

The optional parameter `addons` that we passed to the run command is a ```list``` of EUROMOD  Addons to be integrated in the spine . Each item of the ``list`` is   a ```tuple``` with two ```str``` objects. The first  ```str``` is the name of the Addon and the second `str` is the name of the system in the Addon to be integrated (typically, it is the name of the Addon ```_``` two-letter country code, e.g. LMA_AT).  The default valuye here is []. 

## Run with extensions
Run the simulation for the Swedish system SE_2022 switching on the Benefit Take-up Adjustment extension 'BTA'.

In [20]:
out_BTA =mod['SE']['SE_2022'].run(data,"SE_2021_b1",switches=[("BTA",True)])
out

Simulation for system SE_2022 with dataset SE_2021_b1 finished.


------------------------------
Simulation
------------------------------
	 constantsToOverwrite: {}
	 errors: []
	 output_filenames: ['se_2022_base_mtr.txt', 'se_2022_mtr.txt']
	 outputs: Pandas DataFrame of 246 variables and 21671 observations., Pandas DataFrame of 38 variables and 21671 observations.

In [21]:
out_BTA.outputs[0].ils_ben.mean() - outputdata_baseline.ils_ben.mean()

223.1594797723219

The optional parameter `switches` must define a `list` of the  [EUROMOD](https://euromod-web.jrc.ec.europa.eu "https://euromod-web.jrc.ec.europa.eu") extensions  to be switched on or off in the simulation. Each item in the ``list`` is   a ```tuple``` with two objects. The first object is a ```str``` short name of the Extension. The second object is a ```boolean```. The default is []. 